In [1]:
import cv2 ,time
import numpy as np

In [2]:
def rescale_frame(frame,scale):    # works for image, video, live video
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width-100, height)
    return cv2.resize(frame, dimensions, interpolation=cv2.INTER_AREA)

In [3]:
face_cascade=cv2.CascadeClassifier("cascades/haarcascade_frontalface_alt2.xml")

In [4]:
from tensorflow.keras.models import load_model

In [5]:
model = load_model('model/face_mask_detection_model.h5')

In [6]:
IMG_SIZE = 227

In [7]:
def norm_predictions(predictions):
    norm_pred = []
    pred = []
    
    for i in range(len(predictions)):
        norm_pred.append(np.argmax(predictions[i]))
        
    if norm_pred==0:
        pred.append('Mask')
    else:
        pred.append('Non Mask')
    
    return norm_pred
        

In [8]:
video=cv2.VideoCapture(0)
address = 'http://192.168.0.4:8080//video'
video.open(address)

count = 0

while True:
    check,frame=video.read()
    frame_resized = cv2.resize(frame,(IMG_SIZE,IMG_SIZE))
    frame_rescaled = rescale_frame(frame,0.5)
    gray_frame = cv2.cvtColor(frame_rescaled,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame,scaleFactor=1.5,minNeighbors=5)
    
    frame_exp = np.expand_dims(frame_resized,axis=0)
    
    pred = model.predict(x=frame_exp)
    pred = np.argmax(pred)
    
    if pred==0:
        identify = 'Mask'
    else:
        identify = 'Non Mask'
        
    for x,y,w,h in faces:
        roi_gray = gray_frame[y:y+h,x:x+w]
        stroke = 2
        end_cord_x = x+w #height
        end_cord_y = y+h #width
        
        if pred==0:
            color=(0,255,0)#BGR
        else:
            color=(0,0,255)
            
        cv2.rectangle(frame_rescaled,(x,y),(end_cord_x,end_cord_y),color,stroke)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = identify
        stroke = 2
        
        cv2.putText(frame_rescaled,text,(x+120,y-10),font,1.5,color,stroke)
    

    cv2.imshow('webcam',frame_rescaled)
    key=cv2.waitKey(1)

    if key==ord('w'):
        break   
    

video.release()
cv2.destroyWindow('webcam')